In [33]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import json
from goodreads_scrapper import *

In [3]:
all_isbn = np.load("book_asin_filter.npy",allow_pickle=True).tolist()

In [118]:
to_download = all_isbn
to_download = np.array(to_download)
to_download.sort()

In [221]:
def get_record_from_isbn(isbn):
    
    data = {'q': isbn}
        
    try:
        book_url = get_page("https://www.goodreads.com/search", data)

        soup = bs(book_url.text, 'html.parser')
        record = get_record(soup)
        record["isbn"] = isbn

        time.sleep(1)

    # if searching for isbn is not successful, then search by book title
    except:
        try:
            book_url = get_page("https://www.goodreads.com/search", data)
            soup = bs(book_url.text, 'html.parser')
            url_new = 'https://www.goodreads.com' + soup.find_all('a', class_="bookTitle", href=True)[0]['href']
            headers = Headers().generate()
            time.sleep(1)
            book_url_new = requests.get(url_new, headers=headers, timeout=20)
            soup_new = bs(book_url_new.text, 'html.parser')
            
            record = get_record(soup_new)
            record["isbn"] = isbn
            record["asin"] = True 
            time.sleep(1)
            
        except:
            
            record = {}
            record["isbn"] = isbn
            
    return record

In [136]:
from pqdm.processes import pqdm

In [137]:
# Parallelization
records = pqdm(to_download, get_record_from_isbn, n_jobs=16)

QUEUEING TASKS | :   0%|          | 0/146723 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/146723 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

